In [12]:
# 06_Hotspots_LISA_GiStar_withRoads.py
# Per day:
#   (A) Local Moran's I (LISA) cluster map (HH, LL, HL, LH; p<0.05)
#   (B) Getis–Ord Gi* z-score map (diverging; blue=cold, gray=neutral, red=hot; circles=significant)
# Adds Pasig roads overlay (primary/secondary/tertiary).
# Writes a per-day CSV summarizing LISA cluster counts.
# Robust spatial weights: KNN with k-backoff, fallback to DistanceBand.
# Includes a TEST RUN mode that processes one file, saves *_TEST.png, then optionally proceeds.

import os, glob, re, csv, warnings, sys
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

from shapely.geometry import Point
from fiona.env import Env

# spatial stats
from libpysal.weights import KNN, DistanceBand, fill_diagonal
from esda import Moran_Local, G_Local
from scipy.spatial import cKDTree

# colormaps
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm

# -------- Paths (UPDATE IF NEEDED)
DATA_DIR    = r"C:\Users\HP\Desktop\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP   = r"C:\Users\HP\Desktop\SpatialCARE\Pasig\Pasig.shp"
ROADS_SHP   = r"C:\Users\HP\Desktop\SpatialCARE\Pasig\PasigRN.shp"
OUT_F_DIR   = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\hotspots"
OUT_CSV_DIR = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\hotspot_summaries"
os.makedirs(OUT_F_DIR, exist_ok=True)
os.makedirs(OUT_CSV_DIR, exist_ok=True)

# -------- CRS & figure
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (12, 5.2), 150

# -------- Analysis settings
KNN_K = 8                 # desired neighbors
PERMUTATIONS = 999        # for p-values
ALPHA = 0.05              # significance threshold
VALUE_COL_CANDS = ("pm25","pm_25","pm2_5","pm2.5")

# ---- Exclusion (for analysis ONLY – optional)
EXCLUDE_FLAGGED = True
FLAG_MICRO_SITES = {
    "san antonio barangay hall",
    "san antonio brgy hall",
    "san antonio brgy. hall",
}
def _norm_name(s):
    return re.sub(r"\s+", " ", str(s).strip().lower())

# -------- Road styling (neutral grayscale, transparent)
STYLE_MAP = {
    "primary":   {"linewidth": 2.0, "color": "#555555", "alpha": 0.40},
    "secondary": {"linewidth": 1.6, "color": "#888888", "alpha": 0.30},
    "tertiary":  {"linewidth": 1.2, "color": "#bbbbbb", "alpha": 0.30},
}
ROAD_HANDLES = [
    Line2D([0],[0], color="#555555", lw=2.0, label="Primary"),
    Line2D([0],[0], color="#888888", lw=1.6, label="Secondary"),
    Line2D([0],[0], color="#bbbbbb", lw=1.2, label="Tertiary"),
]

# -------- Cluster colors (categorical; keep as is)
CLUSTER_COLORS = {
    "High–High"      : "#e41a1c",  # strong red
    "Low–Low"        : "#377eb8",  # strong blue
    "High–Low"       : "#984ea3",  # purple
    "Low–High"       : "#4daf4a",  # green
    "Not significant": "#bdbdbd",  # light gray
}

# -------- Gi* colormap (blue → gray → red)
CMAP_BGR = LinearSegmentedColormap.from_list(
    "blue-gray-red", ["#2166ac", "#EDC9AF", "#b2182b"]
)

# -------- Helpers
def pick_val_col(cols):
    for c in cols:
        if str(c).lower() in VALUE_COL_CANDS: return c
    return None

def dedupe_xy(gdf_utm, val_col):
    t = gdf_utm[[val_col,"geometry"]].copy()
    t["x"] = t.geometry.x; t["y"] = t.geometry.y
    t = (t.groupby(["x","y"], as_index=False)[val_col].mean())
    return gpd.GeoDataFrame(t, geometry=gpd.points_from_xy(t["x"], t["y"], crs=gdf_utm.crs))

def lisa_cluster_labels(q, sig_mask):
    # esda.q: 1=HH, 2=LH, 3=LL, 4=HL
    out = np.full(sig_mask.shape[0], "", dtype=object)
    out[(q==1)&sig_mask] = "High–High"
    out[(q==3)&sig_mask] = "Low–Low"
    out[(q==2)&sig_mask] = "Low–High"
    out[(q==4)&sig_mask] = "High–Low"
    out[~sig_mask] = "Not significant"
    return out

def make_weights(xy: np.ndarray, k_desired: int):
    """
    Robust spatial weights:
      1) Try KNN with k=min(k_desired, n-1), backing off if needed
      2) Fallback: DistanceBand using the (k_desired+1)-th neighbor distance as threshold
    Returns (W, info_str)
    """
    n = int(xy.shape[0])
    if n < 3:
        raise ValueError("Not enough points to build spatial weights (n < 3).")

    # Cap k to n-1 (KNN excludes self)
    k_eff = max(1, min(k_desired, n - 1))

    # Try decreasing k to find a valid KNN
    for k in range(k_eff, 1, -1):
        try:
            # ids as list (avoid NumPy-array ambiguity in libpysal)
            w = KNN.from_array(xy, k=k, ids=list(range(n)))
            w.transform = "r"
            return w, f"KNN (k={k})"
        except Exception:
            continue  # back off and try smaller k

    # --- Fallback: DistanceBand ---
    kk = max(2, min(n - 1, k_desired + 1))
    dists = cKDTree(xy).query(xy, k=kk)[0]  # includes self at [:,0]==0
    thr = float(np.nanmax(dists[:, -1]))    # farthest of non-self neighbors
    w = DistanceBand(xy, threshold=thr, binary=False, silence_warnings=True)
    w.transform = "r"
    return w, f"DistanceBand (thr≈{thr:.1f} m)"

# -------- Load boundary & roads
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None:
    roads = roads.set_crs(WGS84)
roads_utm = roads.to_crs(UTM51)

# pick best road attribute
road_col = None
for c in ("highway","fclass","type"):
    if c in roads_utm.columns:
        road_col = c; break
if road_col is None:
    raise SystemExit("No road class column ('highway'/'fclass'/'type') in roads.")
roads_utm[road_col] = roads_utm[road_col].astype(str).str.lower()
roads_sel = roads_utm[roads_utm[road_col].isin({"primary","secondary","tertiary"})].copy()
try:
    roads_clip = gpd.clip(roads_sel, pasig_utm)
except Exception:
    roads_clip = gpd.overlay(roads_sel, pasig_utm[["geometry"]], how="intersection")

# -------- Core processing (single file)
def process_one_file(fpath: str, test_suffix: bool=False):
    base = os.path.splitext(os.path.basename(fpath))[0]
    day  = base.replace("date_", "")

    g = gpd.read_file(fpath)
    if g.crs is None:
        g = g.set_crs(WGS84)

    # ensure geometry exists
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = next((c for c in g.columns if str(c).lower() in ("longitude","lon","x")), None)
        lat = next((c for c in g.columns if str(c).lower() in ("latitude","lat","y")), None)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print(f"Skip (no geometry): {base}")
            return

    val_col = pick_val_col(g.columns)
    if val_col is None:
        print(f"Skip (no PM2.5 col): {base}")
        return

    # Project & clean
    pts = g.to_crs(UTM51)
    pts = dedupe_xy(pts, val_col)
    pts[val_col] = np.clip(pts[val_col].astype(float), 0, None)

    # Optional exclusion (by station name)
    if EXCLUDE_FLAGGED:
        st_col = next((c for c in ("stations","station","Station","STATION") if c in g.columns), None)
        if st_col is not None:
            gg = g.to_crs(UTM51)[[st_col,"geometry"]].copy()
            gg["station_norm"] = gg[st_col].astype(str).map(_norm_name)
            pts = gpd.sjoin(pts, gg[["station_norm","geometry"]], how="left", predicate="intersects")
            pts["is_flagged"] = pts["station_norm"].isin(FLAG_MICRO_SITES)
        else:
            pts["is_flagged"] = False
        pts_use = pts[~pts["is_flagged"]].copy()
    else:
        pts_use = pts.copy()

    n = pts_use.shape[0]
    if n < 4:
        print(f"Skip (n<4): {base}")
        return

    # Coordinates & attribute vector
    xy = np.c_[pts_use.geometry.x.values, pts_use.geometry.y.values]
    z  = pts_use[val_col].values

    # Robust spatial weights
    w, w_info = make_weights(xy, KNN_K)
    print(f"{day}: weights -> {w_info} (n={n})")

    # Local Moran's I (LISA)
    lisa = Moran_Local(z, w, permutations=PERMUTATIONS)
    sig = (lisa.p_sim < ALPHA)
    clusters = lisa_cluster_labels(lisa.q, sig)

    # Getis–Ord Gi* (set diagonal explicitly, suppress warning)
    fill_diagonal(w, 0.5)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=r"Gi\* requested, but .*")
        gi = G_Local(z, w, star=None, permutations=PERMUTATIONS)

    gi_z  = gi.Zs
    gi_sig = (gi.p_sim < ALPHA)

    # ---- Write summary CSV
    out_csv = os.path.join(OUT_CSV_DIR, f"{base}_hotspot_summary.csv")
    uniq, counts = np.unique(clusters, return_counts=True)
    with open(out_csv, "w", newline="", encoding="utf-8") as fcsv:
        wtr = csv.writer(fcsv)
        wtr.writerow(["day","n_points","cluster","count"])
        for u,cnt in zip(uniq, counts):
            wtr.writerow([day, int(n), u, int(cnt)])

    # ---- Plot: two subplots + roads
    fig, (axA, axB) = plt.subplots(1, 2, figsize=FIG_SIZE, dpi=FIG_DPI)

    # (A) LISA cluster map (categorical colors)
    for label, color in CLUSTER_COLORS.items():
        subset = pts_use[clusters == label]
        if not subset.empty:
            subset.plot(ax=axA, color=color, markersize=55,
                        edgecolor="white", linewidth=0.7, zorder=3, label=label)
    pasig_utm.boundary.plot(ax=axA, color="black", linewidth=0.8, zorder=4)
    for klass, style in STYLE_MAP.items():
        sub = roads_clip.loc[roads_clip[road_col] == klass]
        if not sub.empty: sub.plot(ax=axA, **style, zorder=2)  # beneath points
    axA.set_title(f"(A) Local Moran's I — {day}\nClusters at p < {ALPHA}")
    axA.set_axis_off()

    lisa_handles = [mpatches.Patch(color=CLUSTER_COLORS[k], label=k)
                    for k in ["High–High","Low–Low","High–Low","Low–High","Not significant"]]

    # (B) Getis–Ord Gi* z-scores (blue-gray-red, significance outlined)
    vabs = max(2.0, np.nanmax(np.abs(gi_z)))  # symmetric range
    norm = TwoSlopeNorm(vcenter=0.0, vmin=-vabs, vmax=vabs)

    # base points (keep fill by z-score)
    axB.scatter(
        pts_use.geometry.x, pts_use.geometry.y,
        c=gi_z, cmap=CMAP_BGR, norm=norm,
        s=40, edgecolor="white", linewidth=0.4, alpha=0.9, zorder=3
    )

    # significant points: same fill + black outline
    if np.any(gi_sig):
        sig_pts = pts_use.loc[gi_sig]
        axB.scatter(
            sig_pts.geometry.x, sig_pts.geometry.y,
            c=gi_z[gi_sig], cmap=CMAP_BGR, norm=norm,
            s=120, edgecolor="black", linewidth=1.2, zorder=4,
            label="Significant (p<0.05)"
        )

    pasig_utm.boundary.plot(ax=axB, color="black", linewidth=0.8, zorder=2)
    for klass, style in STYLE_MAP.items():
        sub = roads_clip.loc[roads_clip[road_col] == klass]
        if not sub.empty: sub.plot(ax=axB, **style, zorder=1)  # beneath points
    axB.set_title(f"(B) Getis–Ord Gi* — {day}\nz-scores (hot + / cold −)")
    axB.set_axis_off()

    cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=CMAP_BGR), ax=axB, shrink=0.8)
    cb.set_label("Gi* z-score")

    # Bottom legend (roads + LISA classes)
    all_handles = ROAD_HANDLES + lisa_handles
    fig.legend(handles=all_handles, loc="upper center",
               bbox_to_anchor=(0.5, 0.20), ncol=3, frameon=True, fontsize=8, title="Legend")
    plt.subplots_adjust(bottom=0.30, wspace=0.06)

    # output
    suffix = "_TEST" if test_suffix else "_hotspots_combined"
    out_png = os.path.join(OUT_F_DIR, f"{base}{suffix}.png")
    fig.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

    print("Saved:", os.path.basename(out_png), "|", os.path.basename(out_csv))

# -------- Iterate files with TEST mode
TEST_MODE       = True   # True: process only first file then ask to proceed
ASK_TO_PROCEED  = True   # ask y/n after test; only used if TEST_MODE=True

files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

if TEST_MODE:
    test_file = files[0]
    print(f"\n*** TEST RUN on {os.path.basename(test_file)} ***\n")
    process_one_file(test_file, test_suffix=True)

    if ASK_TO_PROCEED:
        try:
            resp = input("\nCheck the TEST figure. Proceed with full run? (yes/no): ").strip().lower()
        except EOFError:
            print("\nNo input available. Exiting after test run.")
            sys.exit(0)
        if resp not in ("y","yes"):
            print("Okay, stopping after test run.")
            sys.exit(0)

    # proceed with remaining files
    print("\nProceeding with full run...\n")
    for f in files:
        if f == test_file:  # skip already processed test file
            continue
        process_one_file(f, test_suffix=False)
else:
    # full run directly
    for f in files:
        process_one_file(f, test_suffix=False)


*** TEST RUN on date_2025-02-06.gpkg ***

2025-02-06: weights -> KNN (k=4) (n=5)


C:\Users\HP\anaconda3\envs\CARESpatial\Lib\site-packages\esda\getisord.py:450: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Gs - self.EG_sim) / self.seG_sim


Saved: date_2025-02-06_TEST.png | date_2025-02-06_hotspot_summary.csv



Check the TEST figure. Proceed with full run? (yes/no):  yes



Proceeding with full run...

2025-02-07: weights -> KNN (k=4) (n=5)


...

Saved: date_2025-07-23_hotspots_combined.png | date_2025-07-23_hotspot_summary.csv
2025-07-24: weights -> KNN (k=4) (n=5)


C:\Users\HP\anaconda3\envs\CARESpatial\Lib\site-packages\esda\getisord.py:450: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Gs - self.EG_sim) / self.seG_sim


Saved: date_2025-07-24_hotspots_combined.png | date_2025-07-24_hotspot_summary.csv
2025-07-25: weights -> KNN (k=5) (n=6)


C:\Users\HP\anaconda3\envs\CARESpatial\Lib\site-packages\esda\getisord.py:450: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Gs - self.EG_sim) / self.seG_sim


Saved: date_2025-07-25_hotspots_combined.png | date_2025-07-25_hotspot_summary.csv
2025-07-26: weights -> KNN (k=6) (n=7)
Saved: date_2025-07-26_hotspots_combined.png | date_2025-07-26_hotspot_summary.csv
2025-07-27: weights -> KNN (k=6) (n=7)
Saved: date_2025-07-27_hotspots_combined.png | date_2025-07-27_hotspot_summary.csv
2025-07-28: weights -> KNN (k=6) (n=7)
Saved: date_2025-07-28_hotspots_combined.png | date_2025-07-28_hotspot_summary.csv
2025-07-29: weights -> KNN (k=6) (n=7)
Saved: date_2025-07-29_hotspots_combined.png | date_2025-07-29_hotspot_summary.csv
2025-07-30: weights -> KNN (k=6) (n=7)
Saved: date_2025-07-30_hotspots_combined.png | date_2025-07-30_hotspot_summary.csv
2025-07-31: weights -> KNN (k=6) (n=7)
Saved: date_2025-07-31_hotspots_combined.png | date_2025-07-31_hotspot_summary.csv


In [13]:
import os
import glob
from PIL import Image

# Folder where your PNG files are located
img_dir = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\hotspots"

# Get all .png files sorted by filename (important for order)
png_files = sorted(glob.glob(os.path.join(img_dir, "*.png")))

# Load images
frames = [Image.open(f) for f in png_files]

# Output file
gif_path = os.path.join(img_dir, "Hotspots_LISA_GiStar.gif")

# Save as GIF
frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=500,   # duration per frame in milliseconds (adjust speed here)
    loop=0          # 0 means infinite loop
)

print(f"GIF saved at: {gif_path}")


GIF saved at: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\hotspots\Hotspots_LISA_GiStar.gif
